# Libs

In [48]:
import re
import requests
import random
import json
import os

from random import randint
from time import sleep
from bs4 import BeautifulSoup
from datetime import datetime

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

# Constants

In [12]:
REGIONS = ["Europe", "CIS", "Americas", "Oceania", "Eastern_%26_Southern_Asia"]
URL = "https://liquipedia.net/counterstrike"
REGION_URL = URL + "/Portal:Players"

# Major aggregation

In [160]:
page = requests.get(URL + "/Majors")
soup = BeautifulSoup(page.content, "html.parser")
event_elements = soup.find_all("div", {"class": "divRow tournament-card-premier"})
major_list = [event.find("b").find("a")["href"] for event in event_elements]
major_list

['/counterstrike/Intel_Extreme_Masters/2022/Rio',
 '/counterstrike/PGL/2022/Antwerp',
 '/counterstrike/PGL/2021/Stockholm',
 '/counterstrike/ESL/One/2020/Rio',
 '/counterstrike/StarLadder/2019/Major',
 '/counterstrike/Intel_Extreme_Masters/Season_XIII/World_Championship',
 '/counterstrike/FACEIT/2018/Major',
 '/counterstrike/ELEAGUE/2018/Major',
 '/counterstrike/PGL/2017/Krakow',
 '/counterstrike/ELEAGUE/2017/Major',
 '/counterstrike/ESL/One/2016/Cologne',
 '/counterstrike/MLG/2016/Columbus',
 '/counterstrike/DreamHack/2015/Cluj-Napoca',
 '/counterstrike/ESL/One/2015/Cologne',
 '/counterstrike/ESL/One/2015/Katowice',
 '/counterstrike/DreamHack/2014/Winter',
 '/counterstrike/ESL/One/2014/Cologne',
 '/counterstrike/ESL/Major_Series_One/2014/Katowice',
 '/counterstrike/DreamHack/2013/Winter']

# Farm

In [175]:
page = requests.get(REGION_URL + "/Europe")
soup = BeautifulSoup(page.content, "html.parser")

In [176]:
flags = [item.find_all("a")[0]["title"] for item in soup.find_all("td")]
elements = [item.find_all("a")[1] for item in soup.find_all("td")]
name_dict = [element["title"] for element in elements]
name_dict

['Stikle-',
 'ChrissK',
 'Dukiiii',
 'Kakafu',
 'NinoZjE',
 'Orbit',
 'ReacTioNNN',
 'S3NSEY',
 'SesL',
 'TWEEZYCS',
 'XTreMe',
 'AntO oNNN',
 'Claeys',
 'FARIS',
 'Infernal',
 'Keoz',
 'KllyVe',
 'Kzealos',
 'MetaL',
 'Monu',
 'Nexius',
 'RitchiEE',
 'Simix',
 'Stev0se',
 'To1nou',
 'YNc',
 'HuNter-',
 'NiKo',
 'Sarenii',
 'Blocker',
 'Bubble',
 'CeRq',
 'Dennyslaw',
 'Doublemagic',
 'Dream3r',
 'Duplicate',
 'H4rn',
 'KalubeR',
 'Mar',
 'Niki1',
 'Numb',
 'Oxygen',
 'Patrick (Bulgarian player)',
 'PNshr',
 'Poizon',
 'Rafftu',
 'Rainwaker',
 'REDSTAR',
 'Rock1nG',
 'ShaiK',
 'SHiPZ',
 'SPELLAN',
 'SpyleadeR',
 'ToH1o',
 'V1c7oR',
 'Zix',
 'AJTT',
 'Beastik',
 'Bfull',
 'Blogg1s',
 'CapseN',
 'Daxen',
 'Desty',
 'DEV7L',
 'Dytor',
 'EYO',
 'Forsyy',
 'Fraged',
 'Fredi',
 'HONES',
 'K1-FiDa',
 'Kinzo',
 'Leckr',
 'Levi',
 'Manguss',
 'MoriiSko',
 'Nbqq',
 'NEOFRAG',
 'Oskar',
 'Pechyn',
 'SHOCK',
 'Stinx',
 'SyncD',
 'Tomkeejs',
 'Twist (Czech player)',
 'Valencio',
 'ZEDKO',
 'Zur1s',

In [188]:
def fetch_player(player_name):
    # pull player profile
    page = requests.get(URL + "/" + player_name)
    soup = BeautifulSoup(page.content, "html.parser")
    
    # parse player information
    info_list = soup.find_all("div", {"class": "infobox-cell-2"})
    info_dict = {info_list[i].text[:-1] : info_list[i+1].text for i in range(0, len(info_list),2)}
    
    # get nationality
    if "Nationality" in info_dict:
        info_dict["Nationality"] = info_dict["Nationality"][1:]
    
    # Get age
    if "Born" in info_dict:
        info_dict["Born"] = datetime.strptime(info_dict["Born"][:-9].replace(",", ""), "%B %d %Y").strftime("%m/%d/%Y")
    
    # Get role
    if "Role" in info_dict:
        if "AWPer" in info_dict["Role"]:
            info_dict["Weapon"] = "AWP"
        else:
            info_dict["Weapon"] = "AK47"
    if "Roles" in info_dict:
        if "AWPer" in info_dict["Roles"]:
            info_dict["Weapon"] = "AWP"
        else:
            info_dict["Weapon"] = "AK47"
            
    # Get major appearances
    page = requests.get(URL + "/" + player_name + "/Results")
    soup = BeautifulSoup(page.content, "html.parser")
    event_elements = soup.find_all("tr", {"class": "valvemajor-highlighted"})
    event_name_list = [event.find("td", {"style": "text-align:left"}).find("a")["href"] for event in event_elements]
    event_name_list = [event for event in event_name_list if event in major_list]
    info_dict["Majors"] = len(event_name_list)
    
    return info_dict
    
fetch_player("electroNic")

{'Name': 'Денис Раилевич Шарипов',
 'Romanized Name': 'Denis Railevich Sharipov',
 'Nationality': 'Russia',
 'Born': '09/02/1998',
 'Status': 'Active',
 'Years Active (Player)': '2015 – Present',
 'Roles': 'In-game leaderRifler',
 'Team': 'Natus Vincere',
 'Alternate IDs': 'electronic',
 'Approx. Total Winnings': '$1,532,094',
 'Games': 'Global Offensive',
 'Weapon': 'AK47',
 'Majors': 8}